In [1]:
# Extract as many features as possible from each word clip and add these cols as X for each word df
# inspired from sunamdha branch code
# check if PCA needed or not and create df's

In [2]:
# change below code to accomodate 3 second clip feature extraction

In [ ]:
# df for Arun
import librosa
import pandas as pd
# import pyrubberband

# Load the audio file
audio, sr = librosa.load("Complete Dataset/Individual Voices - Train/Arunanshu/arun.wav", sr=44100)

# Convert the sample rate to integer
sr = int(sr)

# Calculate the total number of clips
clip_duration = 5
clip_samples = int(sr * clip_duration)
total_clips = len(audio) // clip_samples

# Create the output DataFrame
columns = ["Names"] + [f"MFCC_{i+1}" for i in range(13)] + ["Chroma", "SpectralContrast", "Tonnetz", "ZeroCrossingRate", "RMSEnergy"] + [f"LogMelFilterbank_{i+1}" for i in range(30)]+ ["RASTA-PLP"]
dfArun = pd.DataFrame(columns=columns)
dfArun["Names"] = "Arunanshu"

# Split the audio into clips and extract features for each clip
for i in range(total_clips):
    clip_start = i * clip_samples
    clip_end = (i + 1) * clip_samples
    clip = audio[clip_start:clip_end]

    # Extract MFCCs
    mfccs = librosa.feature.mfcc(y=clip, sr=sr, n_mfcc=13, hop_length=512, n_fft=2048)
    mfccs_flattened = mfccs.mean(axis=1)

    # Extract Chroma features
    chroma = librosa.feature.chroma_stft(y=clip, sr=sr)

    # Extract Spectral Contrast
    contrast = librosa.feature.spectral_contrast(y=clip, sr=sr)

    # Extract Tonnetz features
    tonnetz = librosa.feature.tonnetz(y=clip, sr=sr)

    # Extract Zero Crossing Rate
    zero_crossings = librosa.feature.zero_crossing_rate(y=clip)

    # Extract RMS Energy
    rms_energy = librosa.feature.rms(y=clip)

    # Extract Log Mel Filterbank Energies with 30 mel filterbanks
    mel_filterbank_energies = librosa.feature.melspectrogram(y=clip, sr=sr, n_mels=30)
    log_mel_filterbank_energies = librosa.power_to_db(mel_filterbank_energies)

    # Extract the mean of Log Mel Filterbank Energies along the time axis
    log_mel_filterbank_energies_flattened = log_mel_filterbank_energies.mean(axis=1).tolist()
    
    rasta_plp_features = rasta_plp_feature_extraction(clip, sr)
   # print(np.mean(rasta_plp_features.tolist()))

    # Concatenate all features into a single row
    row_values = ["Arunanshu"] + mfccs_flattened.tolist() + [chroma.mean(), contrast.mean(), tonnetz.mean(), zero_crossings.mean(), rms_energy.mean()] + log_mel_filterbank_energies_flattened + [np.mean(rasta_plp_features.tolist())]


    
    # Append the row to the DataFrame
    dfArun = dfArun._append(pd.Series(row_values, index=dfArun.columns), ignore_index=True)


# Display the resulting DataFrame
dfArun